In [4]:
from sklearn import datasets
import pandas as pd
import numpy as np
import math

In [5]:
# LOAD DATA
iris=datasets.load_iris()

In [6]:
# SEAPRATE X and Y FROM DATA
x_iris=iris.data
y_iris=iris.target

x=np.array([np.array(xi) for xi in x_iris])
y=np.array(y_iris)
print(x.shape)
print(y.shape)

(150, 4)
(150,)


In [7]:
# FEATURE NAMES
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [8]:
# IMPLEMENTATION OF DECISION TREE
decisionTree(x,y,iris.feature_names,0)

level 0
count of  0 ( setosa ) = 50
count of  1 ( versicolor ) = 50
count of  2 ( virginica ) = 50
current entropy= 1.0986122886681096
Splitting on feature petal length (cm) with gain ratio 1.0

level 1
count of  0 ( setosa ) = 50
current entropy=0
Reached leaf node

level 1
count of  1 ( versicolor ) = 50
count of  2 ( virginica ) = 50
current entropy= 0.6931471805599453
Splitting on feature petal width (cm) with gain ratio 0.6621582046482519

level 2
count of  1 ( versicolor ) = 44
count of  2 ( virginica ) = 1
current entropy= 0.10656595882801999
Splitting on feature sepal length (cm) with gain ratio 0.4166719824070529

level 3
count of  1 ( versicolor ) = 1
count of  2 ( virginica ) = 1
current entropy= 0.6931471805599453
Splitting on feature sepal width (cm) with gain ratio 1.0

level 4
count of  1 ( versicolor ) = 1
current entropy=0
Reached leaf node

level 4
count of  2 ( virginica ) = 1
current entropy=0
Reached leaf node

level 3
count of  1 ( versicolor ) = 43
current entrop

In [3]:
# IMPLEMENTED DECISION TREE FUNCTION
def decisionTree(x_train,y_train,features,level):
    # BASE CASES 1 or 2
    # 1) if the node is PURE
    A=set(y_train)
    if len(A)==1:
        print("level",level)
        for i in A:
            cls=i
        print("count of ",cls,"(",iris.target_names[cls],") =",len(y_train))
        print("current entropy=0")
        print("Reached leaf node")
        print()
        return
    
    # 2) if we run out of features
    if len(features)==0:
        print("level",level)
        y_classes=set(y_train)
        for i in y_classes:
            print("count of ",i,"(",iris.target_names[i],") =",(y_train==i).sum())
        print("current entropy=",entropy(x_train,y_train))
        print("Reached leaf node")
        print()
        return
    
    # PROCEEDING FOR RECURSIVE CALLS
    print("level",level) 
    y_classes=set(y_train)
    for i in y_classes:
        print("count of ",i,"(",iris.target_names[i],") =",(y_train==i).sum())
    present_entropy=entropy(x_train,y_train)
    print("current entropy=",present_entropy)
    
    # Feature selection in order to maximize gain
    max_gain=0
    feature_selected=0
    feature_value=0
    for f in range(len(features)):
        # since we have continuous ranged data
        # therefore we need the value at which we decided to split
        current_gain,value = gain_ratio(x_train,y_train,f,features,iris.feature_names,present_entropy)
        if current_gain > max_gain:
            max_gain=current_gain
            feature_selected=f
            feature_value=value
    # if there is no gain, reached leaf node 
    if(max_gain==0):
        print("Reached leaf node")
        print()
        return
    
    # else splitting will occur
    print("Splitting on feature",features[feature_selected],"with gain ratio",max_gain)
    print()
    
    # find column in original data for feature_selected
    cols=0
    for j in range(len(iris.feature_names)):
        if iris.feature_names[j] == features[feature_selected]:
            cols=j
    features.remove(features[feature_selected])
    
    # Recursive call to the split made on feature_selected
    # 1) Calling LEFT NODE by creating new_x_train and new_y_train
    criteria = x_train[:,cols] <= feature_value
    new_x1,new_y1=x_train[criteria],y_train[criteria]
    decisionTree(new_x1,new_y1,features.copy(),level+1)
    
    # 2) Calling RIGHT NODE by creating new_x_train and new_y_train
    criteria = x_train[:,cols] > feature_value
    new_x2,new_y2=x_train[criteria],y_train[criteria]
    decisionTree(new_x2,new_y2,features.copy(),level+1)

In [2]:
def gain_ratio(x,y,f,features,original_features,present_entropy):
    # find column in original data for feature_selected
    cols=0
    for j in range(len(original_features)):
        if original_features[j] == features[f]:
            cols=j
        
    feature_value=set(x[:,cols])
    feature_len=len(feature_value)
    feature_value=sorted(feature_value)
    
    gain_ratio=0
    value=0
    
    # for set of given feature -> iterate over all values -> calculate the maximum gain ratio 
    for i in range(feature_len - 1):
        # choose the midpoint between the 2 values in the feature set
        midpoint=(feature_value[i]+feature_value[i+1])/2
        
        # according to the midpoint -> divide the x_train and y_train -> calculate corresponding entropy
        # values corresponding to less than midpoint
        criteria = x[:,cols] < midpoint
        new_x1,new_y1=x[criteria],y[criteria]
        entropy1=entropy(new_x1,new_y1)
        
        # values corresponding to more than midpoint
        criteria = x[:,cols] >= midpoint
        new_x2,new_y2=x[criteria],y[criteria]
        entropy2=entropy(new_x2,new_y2)
        
        # calculate final entropy by taking in account how much data each side has
        final_entropy=(len(new_y1)/len(y))*entropy1 + (len(new_y2)/len(y))*entropy2
        # calculate the split information
        split_info= (-len(new_y1)/len(y))*math.log(len(new_y1)/len(y)) - (len(new_y2)/len(y))*math.log(len(new_y2)/len(y))
        
        # calculate gain ratio
        current_gain_ratio=(present_entropy-final_entropy)/split_info
        
        # maximize gain ratio for a value in feature
        if(current_gain_ratio > gain_ratio):
            gain_ratio=current_gain_ratio
            value=midpoint
            
    return gain_ratio,value

In [1]:
def entropy(x,y):
    entropy=0
    # calculate total size of y_train
    total_len=len(y)
    y_class=set(y)
    
    # iterate over each class in y_train
    for i in y_class:
        # calculate size of selected class
        current_class_len=(y==i).sum()
        # calculate the probability of a given class 
        probability_current_class=current_class_len/total_len
        # calculate entropy
        entropy+= (-probability_current_class)*math.log(probability_current_class)
    return entropy

In [9]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(criterion='entropy', max_depth=4)
clf.fit(iris.data,iris.target)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
clf.score(iris.data,iris.target)

0.9933333333333333